### Vanilla char-rnn implementation using Numpy

#### Prepare data

In [89]:
import numpy as np

data = open('blog.txt').readlines()
chars = list(''.join(data))

In [90]:
def oneHotEncodeChar(char_pos, input_size):
    vec = np.zeros((1, input_size))
    vec[0][char_pos] = 1
    return vec

In [148]:
def calculateLoss(inputs, targets, prev_hidden_state, Wih, Whh, Who, bh, bo):
    # Make Forward and Backward Pass
    loss = 0
    hidden_state, output_vec, softmax_vec = {}, {}, {} # Save for each time-step to be used in back-prop step
    hidden_state[-1] = prev_hidden_state
        
    # Forward Pass for all inputs collectively
    for t in range(len(inputs)):
        hidden_state[t] = np.tanh(np.dot(inputs[t], Wih) + np.dot(hidden_state[t-1], Whh) + bh) # 1xhidden_size
        output_vec[t] = np.dot(hidden_state[t], Who) + bo # Unnormalized probabilities(1xoutput_size)
        softmax_vec[t] = np.exp(output_vec[t]) / np.sum(np.exp(output_vec[t])) # Softmaxed Probablitites(1xoutput_size)
        loss += -np.log(softmax_vec[t][0][np.where(targets[t] == 1)[0][0]])# Cross Entropy Loss(-y(t)*log(g(t)))
            
    # We first initialize all first order differentials of Weights and Biases w.r.t loss as zeros
    dWih = np.zeros_like(Wih)
    dWhh = np.zeros_like(Whh)
    dWho = np.zeros_like(Who)
    dbh = np.zeros_like(bh)
    dbo = np.zeros_like(bo)
    dhidden_state = np.zeros_like(prev_hidden_state)
    dhidden_state_prev = np.zeros_like(prev_hidden_state)
    
    input_shape = inputs[0].shape
    hidden_shape = prev_hidden_state.shape
    output_shape = targets[0].shape
    
    # Backward-Pass for all inputs in reversed order
    for t in reversed(range(len(inputs))):
        doutput_vec = softmax_vec[t] # 1xoutput_size
        doutput_vec[np.where(targets[t] == 1)[0][0]] -= 1 # TODO: Show Derivation
        assert(doutput_vec.shape == output_shape)
        
        assert(hidden_state[t].shape == hidden_shape)
        dWho += np.dot(hidden_state[t].T, doutput_vec) #hidden_sizexoutput_size
        dbo += doutput_vec# 1xoutput_size
        assert(dWho.shape == Who.shape)
        assert(dbo.shape == bo.shape)
        
        # So the hidden state for last char was only used to calculate output and not used within, i.e. we initialize dhidden_state_prev as 0
        dhidden_state = np.dot(doutput_vec[np.where(targets[t] == 1)[0][0]].T, Who.T) + dhidden_state_prev # Since hidden state flows through output and within itself 
        assert(dhidden_state.shape == hidden_shape)
        
        dtanh = (1-hidden_state[t]**2)*dhidden_state
        assert(dtanh.shape == hidden_shape)
        
        dWhh += np.dot(hidden_state[t-1].T, dtanh)
        dbh += dtanh
        dWih += np.dot(inputs[t].T, dtanh)
        assert(dWhh.shape == Whh.shape)
        assert(dbh.shape == bh.shape)
        assert(dWih.shape == Wih.shape)
        
        dhidden_state_prev = np.dot(dtanh, Whh) # So if this backward pass is for last char, this dhidden_state_prev is for second last char
        
    # Taking care of exploding gradients. Making sure that they are neither too big nor too small, 
    # since this might be a problem with tanh activation function
    for dparam in [dWih, dWhh, dWho, dbh, dbo]:
        np.clip(dparam, -3, 3, out=dparam)
    return loss, hidden_state[len(inputs)-1], dWih, dWhh, dWho, dbh, dbo

In [149]:
def charRNN(inputs, hidden_size=50, batch_chars=20):
    # Both input and output are one-hot encoded vectors
    input_size = len(inputs)
    output_size = input_size
    
    # Create dict for char-to-index mapping
    #char_to_index = {ch:i for i, ch in enumerate(inputs)}
    #index_to_char = {i:ch for i, ch in enumerate(inputs)}
    
    # Initialize weights(from a normal distribution with mean = 0.01) and biases
    Wih = np.random.randn(input_size, hidden_size)*0.01
    Whh = np.random.randn(hidden_size, hidden_size)*0.01
    Who = np.random.randn(hidden_size, output_size)*0.01
    bh = np.zeros((1, hidden_size))
    bo = np.zeros((1, output_size))
    
    # Iterate over input_chars(till second last char), 'batch_chars' at a time
    position=0
    iterations = 0
    prev_hidden_state = np.zeros((1, hidden_size)) # Initialize hidden state to all zeros
    
    # Initialize all momentum terms for Gradient Descent
    momWih = np.zeros_like(Wih)
    momWhh = np.zeros_like(Whh)
    momWho = np.zeros_like(Who)
    mombh = np.zeros_like(bh)
    mombo = np.zeros_like(bo)
    
    learn_rate = 0.01
    mom_rate = 0.9
    
    while(position < input_size-1):
        sub_list_end = position + batch_chars
        if(sub_list_end >= input_size-1):
            sub_list_end = input_size-2 # Till second last char

        if(position == sub_list_end): break
        
        input_sub_list = range(position, sub_list_end)
        targets = range(position+1, sub_list_end+1) # Next char in the sequence is our desired output
        
        inputs_hot_encoded = []
        targets_hot_encoded = []
        for char_pos in input_sub_list: inputs_hot_encoded.append(oneHotEncodeChar(char_pos, input_size))
        for char_pos in targets: targets_hot_encoded.append(oneHotEncodeChar(char_pos, input_size))

        loss, prev_hidden_state, dWih, dWhh, dWho, dbh, dbo = calculateLoss(inputs_hot_encoded, targets_hot_encoded, 
                                                                 prev_hidden_state, Wih, Whh, Who, bh, bo)
        
        print("Iteration#" + str(iterations) + ", Loss=" + str(loss))
        
        # Momentum based Gradient-Descent
        combined = zip([Wih, Whh, Who, bh, bo], [dWih, dWhh, dWho, dbh, dbo], [momWih, momWhh, momWho, mombh, mombo])
        for param, dparam, momparam in combined:
            momparam = mom_rate*momparam + learn_rate*dparam
            param -= momparam

        position = sub_list_end
        iterations += 1

In [150]:
charRNN(chars)

Iteration#0, Loss=176.897075136
Iteration#1, Loss=176.852691298
Iteration#2, Loss=176.853433021
Iteration#3, Loss=176.021581137
Iteration#4, Loss=175.307190448
Iteration#5, Loss=175.2153371
Iteration#6, Loss=175.260281088
Iteration#7, Loss=175.265903101
Iteration#8, Loss=175.266191006
Iteration#9, Loss=175.266201871
Iteration#10, Loss=175.266200174
Iteration#11, Loss=175.266199034
Iteration#12, Loss=175.266198236
Iteration#13, Loss=175.266197613
Iteration#14, Loss=175.26619713
Iteration#15, Loss=175.266196766
Iteration#16, Loss=175.266196447
Iteration#17, Loss=175.26619621
Iteration#18, Loss=175.266196008
Iteration#19, Loss=175.266195852
Iteration#20, Loss=175.266195707
Iteration#21, Loss=175.266195591
Iteration#22, Loss=175.266195496
Iteration#23, Loss=175.266195411
Iteration#24, Loss=175.266195336
Iteration#25, Loss=175.266195275
Iteration#26, Loss=175.266195213
Iteration#27, Loss=175.266195156
Iteration#28, Loss=175.266195114
Iteration#29, Loss=175.266195073
Iteration#30, Loss=175.2